In [1]:
import torch
from torch import nn, einsum
def split_at_index(dim, index, t):
    pre_slices = (slice(None),) * dim
    l = (*pre_slices, slice(None, index))
    r = (*pre_slices, slice(index, None))
    return t[l], t[r]

def exists(val):
    return val is not None

def default(value, d):
    return d if not exists(value) else value

In [27]:
# どうやら本当にlinear attentionをやっていそう。
# ただし、関数φは論文のものと違うものを利用していそう。
def causal_linear_attn(q, k, v, kv_mask = None, bucket_size = None, eps = 1e-6):
    b, h, n, e, dtype = *q.shape, q.dtype
    bucket_size = default(bucket_size, 64)
    bucket_size = max(bucket_size, 1)
    assert bucket_size == 0 or (n % bucket_size) == 0, f'sequence length {n} must be divisible by the bucket size {bucket_size} for causal linear attention'
    
    ## 論文のφにあたる非線形関数
    q = q.softmax(dim=-1)
    k = torch.exp(k).type(dtype).clone()

    q = q * e ** -0.5
    ## ここまで。
    ## ちょっと違う。。。。

#     if exists(kv_mask):
#         mask = kv_mask[:, None, :, None]
#         k = k.masked_fill_(~mask, 0.)
#         v = v.masked_fill_(~mask, 0.)
#         del mask

    bucket_fn = lambda x: x.reshape(*x.shape[:-2], -1, bucket_size, e)# batch, headを固定、時間方向を調整してbucketをつくる
                                                                                                      # いい感じに時間をbucket_sizeごとに区切る
    b_q, b_k, b_v = map(bucket_fn, (q, k, v))

    b_k_sum = b_k.sum(dim=-2) # bucket_size = 1 の時、b_k_sum == kになる。
    b_k_cumsum = b_k_sum.cumsum(dim = -2).type(dtype)# 時間方向の累積和

    context = einsum('bhund,bhune->bhude', b_k, b_v)
    context = context.cumsum(dim = -3).type(dtype)# 時間方向に累積和

#     if bucket_size > 1:
#         context = F.pad(context, (0, 0, 0, 0, 1, 0), value = 0.)
#         context, _ = split_at_index(2, -1, context)

#         b_k_cumsum = F.pad(b_k_cumsum, (0, 0, 1, 0), value = 0.)
#         b_k_cumsum, _ = split_at_index(2, -1, b_k_cumsum)

    D_inv = 1. / einsum('bhud,bhund->bhun', b_k_cumsum + eps, b_q)
    attn = einsum('bhund,bhude,bhun->bhune', b_q, context, D_inv)
    import pdb; pdb.set_trace()
    return attn.reshape(*q.shape)# bucket_sizeをもとに戻してreturn

In [28]:
B, H, N, D = 3, 4, 10, 16

q = torch.rand(B, H, N, D)
k = torch.rand(B, H, N, D)
v = torch.rand(B, H, N, D)

In [29]:
causal_linear_attn(q, k, v, bucket_size=1).shape

> <ipython-input-27-a47355b9a274>(38)causal_linear_attn()
     34 
     35     D_inv = 1. / einsum('bhud,bhund->bhun', b_k_cumsum + eps, b_q)
     36     attn = einsum('bhund,bhude,bhun->bhune', b_q, context, D_inv)
     37     import pdb; pdb.set_trace()
---> 38     return attn.reshape(*q.shape)

ipdb> attn.shape
torch.Size([3, 4, 10, 1, 16])
ipdb> p q.shape
torch.Size([3, 4, 10, 16])
ipdb> q


BdbQuit: 

In [7]:
a = torch.rand(3,4,5,5)
torch.tril(a)

tensor([[[[0.7670, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.3722, 0.3847, 0.0000, 0.0000, 0.0000],
          [0.1686, 0.1331, 0.8143, 0.0000, 0.0000],
          [0.5201, 0.9998, 0.7648, 0.5760, 0.0000],
          [0.7237, 0.6929, 0.8208, 0.2476, 0.0573]],

         [[0.5167, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.4855, 0.6765, 0.0000, 0.0000, 0.0000],
          [0.9214, 0.0434, 0.1406, 0.0000, 0.0000],
          [0.7855, 0.0215, 0.2152, 0.0228, 0.0000],
          [0.5312, 0.2002, 0.1534, 0.8580, 0.7065]],

         [[0.5660, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.7247, 0.6455, 0.0000, 0.0000, 0.0000],
          [0.1308, 0.8853, 0.9953, 0.0000, 0.0000],
          [0.2020, 0.6613, 0.0810, 0.0269, 0.0000],
          [0.8137, 0.5448, 0.9166, 0.7249, 0.6837]],

         [[0.0574, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.7372, 0.7714, 0.0000, 0.0000, 0.0000],
          [0.8085, 0.1648, 0.6225, 0.0000, 0.0000],
          [0.0596, 0.0167, 0.7961, 0.7025, 0.0000],
      

In [5]:
B, h, N_q, d, N_k = 3, 4, 32, 256, 64
q = torch.rand(B, h, N_q, d)
k = torch.rand(B, h, N_k, d)
torch.einsum("bhqd,bhkd->bhqk", q, k).shape

torch.Size([3, 4, 32, 64])

In [9]:
import torch.nn.functional as F
data = torch.rand(3, 5, 2)
F.softmax(data, dim=-1)

tensor([[[0.3551, 0.6449],
         [0.4360, 0.5640],
         [0.6850, 0.3150],
         [0.4151, 0.5849],
         [0.6779, 0.3221]],

        [[0.5647, 0.4353],
         [0.6017, 0.3983],
         [0.4974, 0.5026],
         [0.6586, 0.3414],
         [0.4476, 0.5524]],

        [[0.5124, 0.4876],
         [0.4971, 0.5029],
         [0.3747, 0.6253],
         [0.4854, 0.5146],
         [0.3768, 0.6232]]])